In [1]:
from IPython.display import display

import os
import yaml

import numpy as np
import pandas as pd

from utils import (
    highlight_nan, highlight_dtype, duplicate_columns, generate_stats,
)

## Read data

In [2]:
df = pd.read_csv("../data/loan.csv", nrows=100_000)
display(df.shape)
display(df.head(10))

/vols/build/cms/sdb15/miniconda2/envs/torch38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (123,124,125,128,129,130,133,139,140,141) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(100000, 145)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,5550,5550,5550,36 months,15.02,192.45,C,C3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,2000,2000,2000,36 months,17.97,72.28,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,6000,6000,6000,36 months,13.56,203.79,C,C1,...,NaN,NaN,DirectPay,N,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,5000,5000,5000,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,6000,6000,6000,36 months,14.47,206.44,C,C2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


### Metadata

View description of each feature, alone with their assigned data type, 1st and 2nd
values and the number of unique entires.

In [3]:
info = pd.concat([
    pd.read_excel(
        "../data/LCDataDictionary.xlsx", "LoanStats",
        names=["name", "desc"],
    ),
    pd.read_excel(
        "../data/LCDataDictionary.xlsx", "browseNotes",
        names=["name", "desc"],
    ),
]).dropna().reset_index(drop=True)

# remove whitespace and duplicates
info["name"] = info["name"].str.strip()
info = info.loc[~info["name"].duplicated()]

# sort features
info = info.set_index("name")
info.index = pd.Categorical(info.index, df.columns)
info = info.sort_index()

# add additional info
info["dtype"] = df.dtypes
info["1st"] = df.iloc[0,:]
info["2nd"] = df.iloc[1,:]
info["nuniq"] = df.nunique()

info = info[["dtype", "1st", "2nd", "nuniq", "desc"]]
info = info.reset_index()
info = info.loc[~info["index"].isna()]

# print
info.style.set_properties(subset=["desc"], **{"width": '1000px'})

,index,dtype,1st,2nd,nuniq,desc
0,id,float64,nan,nan,0.000000,A unique LC assigned ID for the loan listing.
1,member_id,float64,nan,nan,0.000000,A unique LC assigned Id for the borrower member.
2,loan_amnt,int64,2500,30000,1463.000000,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
3,funded_amnt,int64,2500,30000,1463.000000,The total amount committed to that loan at that point in time.
4,funded_amnt_inv,int64,2500,30000,1483.000000,The total amount committed by investors for that loan at that point in time.
5,term,object,36 months,60 months,2.000000,The number of payments on the loan. Values are in months and can be either 36 or 60.
6,int_rate,float64,13.560000,18.940000,46.000000,Interest Rate on the loan
7,installment,float64,84.920000,777.230000,10586.000000,The monthly payment owed by the borrower if the loan originates.
8,grade,object,C,D,7.000000,LC assigned loan grade
9,sub_grade,object,C1,D2,35.000000,LC assigned loan subgrade


In [4]:
set(df.columns) - set(info["index"])

{'verification_status_joint'}

This feature has no description and hence is dropped

## Problematic features

In [5]:
bad_cols = set([
    # redundant with grade
    # see: https://www.lendingclub.com/investing/investor-education/interest-rates-and-fees
    "int_rate", "sub_grade",
    
    # redundant with purpose
    "title",
    
    # redundant with addr_state
    "zip_code",
    
    # always NaN
    "id", "member_id", "url", "desc",
    
    # contain future information
    "funded_amnt", "funded_amnt_inv", "issue_d", "out_prncp", "out_prncp_inv",
    "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "recoveries",
    "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt",
    "next_pymnt_d",
    
    # no information
    "verification_status_joint",
    
    # only valid for some accounts. Will drop these for convenience
    # For joint account holders, this could be handled by assuming non-joint
    # accounts have a secondary person with exactly the same merits as the
    # first person
    "sec_app_chargeoff_within_12_mths", "sec_app_collections_12_mths_ex_med",
    "sec_app_earliest_cr_line", "sec_app_inq_last_6mths", "sec_app_mort_acc",
    "sec_app_mths_since_last_major_derog", "sec_app_num_rev_accts",
    "sec_app_open_acc", "sec_app_revol_util", "annual_inc_joint", "dti_joint",
])

## Loan status

The goal is to predict the `loan_status` of a borrower. This feature has the following values:

In [6]:
df["loan_status"].value_counts()

Current               96793
Fully Paid             2431
Late (31-120 days)      318
In Grace Period         316
Late (16-30 days)       123
Charged Off              19
Name: loan_status, dtype: int64

We are interested in the fully paid and charged off loan. The late loans may become charged off.

## Reload data

In [7]:
path = "../data/loan_filtered.csv"
if not os.path.exists(path):
    df = pd.read_csv("../data/loan.csv", low_memory=False)
    df = df.loc[df["loan_status"].isin(["Charged Off", "Fully Paid"])]
    df = df.drop(bad_cols, axis=1)
    df.to_csv(path, index=False)
df = pd.read_csv(path, low_memory=False)

Some categorical types contain NaN which we will set to "Unknown" as the value.

Similarly some floating values related to delinquency do not affect non-delinquents, hence their values will be set at infinity instead of NaN (or a large number).

Furthermore, some entries with no values such as total current balance can be assumed to be zero to stay on the safe side.

In [8]:
for col in ["emp_title", "emp_length"]:
    df.loc[df[col].isna(),col] = "Unknown"

    
for col in [
    "mths_since_last_delinq", "mths_since_last_record",
    "mths_since_last_major_derog",
]:
    df.loc[df[col].isna(),col] = 10000
    
    
for col in ["tot_coll_amt", "tot_cur_bal"]:
    df.loc[df[col].isna(),col] = 0.

In [9]:
stats = generate_stats(df)
stats.T.style.apply(highlight_nan, axis=0).apply(highlight_dtype, axis=0)

Features with more than 1% of entries as NaNs will be dropped. Afterwards, rows with remaining NaNs will be dropped

In [10]:
nan_cols = stats.query("nan > 0.01*rows").index
df = df.drop(nan_cols, axis=1).dropna()

In [11]:
stats = generate_stats(df)
stats.T.style.apply(highlight_nan, axis=0).apply(highlight_dtype, axis=0)

,loan_amnt,term,installment,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,total_pymnt_inv,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,hardship_flag,disbursement_method,debt_settlement_flag
dtype,int64,object,float64,object,object,object,object,float64,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,float64,int64,float64,float64,object,float64,float64,object,float64,float64,int64,object,float64,float64,float64,float64,float64,float64,float64,object,object,object
rows,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732
count,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732
nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
inf,0.000000,,0.000000,,,,,0.000000,,,,,,0.000000,0.000000,,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,,0.000000,0.000000,,0.000000,0.000000,0.000000,,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,,,
mean,14420.216220,,438.175459,,,,,76168.232298,,,,,,18.265112,0.317525,,0.659160,5066.093561,8314.423974,11.598571,0.215737,16242.249561,51.916405,25.026980,,14795.465705,14775.382427,,0.017007,7383.934808,1.000000,,0.005072,236.079721,133814.596396,0.009083,15.011648,0.134774,0.052042,,,
std,8698.331441,,261.026237,,,,,69970.114453,,,,,,10.939776,0.877068,,0.941022,4982.611903,3727.644848,5.463403,0.603182,22279.922594,24.497135,11.995221,,10327.900508,10323.934186,,0.146556,4382.087237,0.000000,,0.077455,10931.130738,156287.132863,0.109985,816.082749,0.378378,0.399010,,,
min,500.000000,,4.930000,,,,,16.000000,,,,,,-1.000000,0.000000,,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,,0.000000,0.000000,,0.000000,0.000000,1.000000,,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,,,
25%,8000.000000,,249.010000,,,,,45996.000000,,,,,,11.800000,0.000000,,0.000000,31.000000,10000.000000,8.000000,0.000000,5961.000000,33.600000,16.000000,,6827.727500,6812.007500,,0.000000,77.000000,1.000000,,0.000000,0.000000,25175.000000,0.000000,0.000000,0.000000,0.000000,,,
50%,12000.000000,,375.430000,,,,,65000.000000,,,,,,17.610000,0.000000,,0.000000,10000.000000,10000.000000,11.000000,0.000000,11147.000000,52.300000,23.000000,,12118.820000,12100.385000,,0.000000,10000.000000,1.000000,,0.000000,0.000000,70197.000000,0.000000,0.000000,0.000000,0.000000,,,


## Data types

### Numeric

In [12]:
int_cols = set(stats.loc[stats["dtype"].str.contains("int")].index)
float_cols = set(stats.loc[stats["dtype"].str.contains("float")].index)

### Date/time

All dates are labelled with a month and year. This can be converted into a number of months since 1970 (unix time) to be used as an integer: (year - 1970)*12 + (month - 1)

In [13]:
def contains_month(x):
    result = np.zeros_like(x, dtype='bool')
    try:
        result = x.str.contains("^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)-[0-9][0-9][0-9][0-9]$")
    except AttributeError:
        pass
    return result

dates = df.apply(contains_month, axis=0).any()
date_cols = set(dates[dates].index)

/vols/build/cms/sdb15/miniconda2/envs/torch38/lib/python3.8/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [14]:
dates = []
for dcol in date_cols:
    date = df[dcol].astype("datetime64")
    dates.append(((date.dt.year-1970)*12 + (date.dt.month - 1)))
dates = pd.concat(dates, axis=1)

df.loc[:,date_cols] = dates[date_cols]

### Target feature

In [15]:
df["loan_status"] = (df["loan_status"]=="Charged Off")

Find any duplicate columns to drop.

In [16]:
tdf = df[int_cols]
dup_cols = set([dup_l for dup_l, dup_r in duplicate_columns(tdf)])

tdf = df[float_cols]
dup_cols |= set([dup_l for dup_l, dup_r in duplicate_columns(tdf)])

tdf = df[date_cols]
dup_cols |= set([dup_l for dup_l, dup_r in duplicate_columns(tdf)])

cat_cols = set(df.columns) - int_cols - float_cols - date_cols
tdf = df[cat_cols]
dup_cols |= set([dup_l for dup_l, dup_r in duplicate_columns(tdf)])
dup_cols

set()

### Categoricals

Some categoricals have an order and these will be encoded as integers

In [17]:
display(df["emp_length"].unique())

array(['5 years', '< 1 year', '10+ years', '3 years', '4 years', '1 year',
       '8 years', 'Unknown', '2 years', '6 years', '9 years', '7 years'],
      dtype=object)

In [18]:
df["emp_length"] = df["emp_length"].replace({
    "Unknown": -1, "< 1 year": 0, "1 year": 1, "2 years": 2, "3 years": 3,
    "4 years": 4, "5 years": 5, "6 years": 6, "7 years": 7, "8 years": 8,
    "9 years": 9, "10+ years": 10,
})
display(df["emp_length"].unique())

array([ 5,  0, 10,  3,  4,  1,  8, -1,  2,  6,  9,  7])

In [19]:
display(df["grade"].unique())

array(['D', 'C', 'A', 'B', 'E', 'G', 'F'], dtype=object)

In [20]:
df["grade"] = df["grade"].replace({
    "A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7,
})
display(df["grade"].unique())

array([4, 3, 1, 2, 5, 7, 6])

Other categoricals don't have an order and we need one-hot encoding. `emp_title` has too many unique values and hence is inefficient for one hot encoding and hence will be dropped. `addr_state` also have a few too many unique values.

In [21]:
df = df.drop(["emp_title", "addr_state"], axis=1)

In [22]:
cat_cols -= set(["emp_title", "addr_state", "emp_length", "grade", "loan_status"])

In [23]:
for col in cat_cols:
    df = pd.concat([
        df, pd.get_dummies(df[col], prefix=col, drop_first=True)
    ], axis=1)
    df = df.drop(col, axis=1)

View metadata information once again

In [24]:
stats = generate_stats(df, nunique=True)
stats.T.style.apply(highlight_nan, axis=0).apply(highlight_dtype, axis=0)

,loan_amnt,installment,grade,emp_length,annual_inc,loan_status,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,term_ 60 months,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y
dtype,int64,float64,int64,int64,float64,bool,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8
rows,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732
count,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732,1301732
nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
inf,0.000000,0.000000,0.000000,0.000000,0.000000,,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,,,,,,,,,,,,,,,,,,,,,,,,,,
mean,14420.216220,438.175459,2.751137,5.568171,76168.232298,,18.265112,0.317525,349.566554,0.659160,5066.093561,8314.423974,11.598571,0.215737,16242.249561,51.916405,25.026980,14795.465705,14775.382427,579.379483,0.017007,7383.934808,1.000000,0.005072,236.079721,133814.596396,0.009083,15.011648,0.134774,0.052042,,,,,,,,,,,,,,,,,,,,,,,,,,
std,8698.331441,261.026237,1.296849,3.932612,69970.114453,,10.939776,0.877068,91.103797,0.941022,4982.611903,3727.644848,5.463403,0.603182,22279.922594,24.497135,11.995221,10327.900508,10323.934186,13.342308,0.146556,4382.087237,0.000000,0.077455,10931.130738,156287.132863,0.109985,816.082749,0.378378,0.399010,,,,,,,,,,,,,,,,,,,,,,,,,,
min,500.000000,4.930000,1.000000,-1.000000,16.000000,,-1.000000,0.000000,-429.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,451.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,,,,,,,,,,,,,,,,,,,,,,,,,,
25%,8000.000000,249.010000,2.000000,2.000000,45996.000000,,11.800000,0.000000,303.000000,0.000000,31.000000,10000.000000,8.000000,0.000000,5961.000000,33.600000,16.000000,6827.727500,6812.007500,574.000000,0.000000,77.000000,1.000000,0.000000,0.000000,25175.000000,0.000000,0.000000,0.000000,0.000000,,,,,,,,,,,,,,,,,,,,,,,,,,
50%,1200

Find columns which do not have enough unique values.

In [25]:
notuniq_cols = set(stats.query("nuniq<2").index)
df = df.drop(notuniq_cols, axis=1)
notuniq_cols

{'policy_code'}

## Export data

In [26]:
df.to_csv("../data/loan_cleaned.csv", index=False)